Create evaluations of the likelihood for plotting purposes.
Evaluate likelihood on a grid of prm, ic combinations.

In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [2]:
using DifferentialEquations, CSV, DataFrames, Plots, Distributions

In [3]:
function klausmeier!(du,u,p,t)
 du[1] = -u[1] - u[1] * u[2]^2 + p[1]
 du[2] = u[1] * u[2]^2 - p[2] * u[2]
end

klausmeier! (generic function with 1 method)

In [4]:
abstract type AbstractHyperprm end

struct Hyperprm <: AbstractHyperprm
    w0::Float64
    n0::Float64
    a::Float64
    m::Float64
    M::Int64
    noise::Float64
end

In [5]:
abstract type AbstractGridprm end

struct Gridprm <: AbstractGridprm
    x_end::Float64
    y_end::Float64
    len::Int64
end

In [6]:
function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

sol_klausmeier (generic function with 1 method)

In [ ]:
function create_grid(gprm::Gridprm)
    x_vals = range(0, gprm.x_end; length=gprm.len)
    y_vals = range(0, gprm.y_end; length=gprm.len)
    
    grid = [(x, y) for x in x_vals, y in y_vals] 
    return grid
end

create_grid (generic function with 1 method)

In [8]:
function randomize_data(df::DataFrame, noise::Float64)
    if noise == 0.0
        return df
    else
        df[!, "w"] .= df[!, "w"] .+ rand(Normal(0, noise), nrow(df))
        df[!, "n"] .= df[!, "n"] .+ rand(Normal(0, noise), nrow(df))
        return df
    end
end

randomize_data (generic function with 1 method)

In [9]:
function compute_ll(true_val::DataFrame, pred_val::DataFrame) # Q: consider difference for n trajectory only? 
    return -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2)
end

compute_ll (generic function with 1 method)

In [10]:
function store_ll_data(w0::Float64,n0::Float64,a::Float64,m::Float64,M::Int64,noise::Float64,df::DataFrame)
    CSV.write("c:/Users/lisah/Documents/Repos/ma-code/data/likelihood/m0.45/$(w0)_$(n0)_$(a)_$(m)_$(M)_$(noise).csv", df)
end

store_ll_data (generic function with 1 method)

In [11]:
function gen_ll_evals_for_hprm_comb(hprm_true::Hyperprm, gprm::Gridprm)

    grid = create_grid(gprm)
    sol_true = sol_klausmeier(hprm_true) # returns df
    sol_true = randomize_data(sol_true, hprm_true.noise) # include noise

    ll = zeros(gprm.len,gprm.len)

    for i in range(1, gprm.len)
        for j in range(1, gprm.len)
            #eval model for each point on grid
            pt = grid[i,j]
            hprm = Hyperprm(hprm_true.w0, pt[2], pt[1], hprm_true.m, hprm_true.M, hprm_true.noise) #w0,n0,a,m,M
            sol_pred = sol_klausmeier(hprm)

            #eval likelihood
            ll_val = compute_ll(sol_true, sol_pred)
            
            ll[i,j] = ll_val
        end
    end
    
    #return data frame
    a_eval_pts = string.(collect(range(0, gprm.x_end; length=gprm.len)))
    df_ll = DataFrame(ll, a_eval_pts)

    return df_ll
end

gen_ll_evals_for_hprm_comb (generic function with 1 method)

Consider a in (0,2), n0 in (0,4).
As in the reference paper we evaluate 11 grid points.

In [17]:
prm_eval_grid = Gridprm(2.0, 4.0, 11)

Gridprm(2.0, 4.0, 11)

The exact prm combinations to consider are chosen similarly to the reference paper and result in the following values. we are only interested in 5 index combinations, namely [1,1], [2,4], [3,5], [4,2], [4,3]

In [ ]:
a_vals = [0.1, 0.9, 1.1, 1.5] 
n0_vals = [0.4, 0.5, 1.0, 1.2, 2.3]

index_combos = [[1,1], [2,4], [3,5], [4,2], [4,3]]

5-element Vector{Vector{Int64}}:
 [1, 1]
 [2, 4]
 [3, 5]
 [4, 2]
 [4, 3]

We try sample sizes M = [10, 100, 500, 1000], and noises = [0.0, 0.1, 1.0]

In [25]:
M_vals = [10, 100, 500, 1000]
noise_vals = [0.0, 0.1, 1.0]

3-element Vector{Float64}:
 0.0
 0.1
 1.0

For now we fix the mortality rate as in the literature and set w0 = 0.0

In [26]:
m = 0.45
w0 = 0.0

0.0

In [27]:
ind = [2,4]
M = 1000
noise = 0.0

a_ind = ind[1]
n0_ind = ind[2]
hprm = Hyperprm(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise)
df_ll = gen_ll_evals_for_hprm_comb(hprm, prm_eval_grid)
 
store_ll_data(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise, df_ll)
#print(df_ll)

"c:/Users/lisah/Documents/Repos/ma-code/data/likelihood/m0.45/0.0_1.3_0.9_0.45_1000_0.0.csv"

? how store data (filename)

In [29]:
# generate data for all hyperparameter combinations
# for ind in index_combos
#     for M in M_vals
#         for noise in noise_vals
#             a_ind = ind[1]
#             n0_ind = ind[2]
#             hprm = Hyperprm(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise)
#             df_ll = gen_ll_evals_for_hprm_comb(hprm, prm_eval_grid)

#             store_ll_data(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise, df_ll)
#         end
#     end
# end